In [1]:
import os
import sys
sys.path.insert(1, '../../../..')

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.FNO import FNO1d
from methods.methodsDataset.FNODataset import FNODataset
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1., 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_FNO = {'layer_dims': None, 'FourierLayers_modes': None, 'device': device, 'seed': 123}

In [4]:
Dmin, Dmax = 0.1, 10
D_list = np.linspace(Dmin, Dmax, 1000)

U_sols = []
solver = Solver(params={'solver': params_solver})
for d in D_list:
    solver.change_D(new_D=d)
    U_sols.append(solver.solve())

U_sols = np.stack(U_sols)
D = torch.Tensor(D_list).view(-1, 1).to(device)
U = torch.Tensor(U_sols).to(device)

In [5]:
D_train, D_test, U_train, U_test = sklearn.model_selection.train_test_split(D, U,test_size=0.2, random_state=params_FNO['seed'])

In [6]:
def get_dataset():
    return D_train, D_test, U_train, U_test

In [7]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [8]:
def define_model(trial, input_chanel):
    n_fourier_layers = trial.suggest_int("n_fourier_layers", 1, 5)
    lifting_layer = trial.suggest_int("lifting_layer", 32, 64)
    layers = [input_chanel, lifting_layer]
    fourier_modes = []
    for i in range(n_fourier_layers):
        out_features = trial.suggest_int("Fourier_units_l{}".format(i), 32, 64)
        fourier_mode = trial.suggest_int("Fourier_modes_l{}".format(i), 12, 16)
        layers += [out_features]
        fourier_modes.append(fourier_mode)
    
    params_FNO_trial = copy.deepcopy(params_FNO)
    params_FNO_trial['layers_dim'] = layers
    params_FNO_trial['FourierLayers_modes'] = fourier_modes
    return FNO1d(params={'solver':params_solver, 'method':params_FNO_trial})

In [9]:
def objective(trial):
    # Generate the model.
    model = define_model(trial, 1)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    D_train, D_val, U_train, U_val = get_dataset()

    trainDataset = FNODataset(x=D_train, y=U_train)
    valDataset = FNODataset(x=D_val, y=U_val)

    batch_size = trial.suggest_int("batch_size", 32, 256)
    torch.manual_seed(params_FNO['seed'])
    trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
    valLoader = torch.utils.data.DataLoader(valDataset, batch_size=batch_size, shuffle=False)

    # Training of the model.
    loss_vals = []
    for epoch in range(2000):
        model.train()
        for i, data in enumerate(trainLoader):
            inputs, label = data
            optimizer.zero_grad()
            output = model(inputs).squeeze(-1)
            loss = loss_fn(output, label)
            loss.backward()
            optimizer.step()
            
        # Validation of the model.
        model.eval()
        loss_val = 0.
        with torch.no_grad():
            for i, data in enumerate(valLoader):
                inputs, label = data
                optimizer.zero_grad()
                output = model(inputs).squeeze(-1)
                loss_val += loss_fn(output, label).item()
        loss_val /= (i+1)
        loss_vals.append(loss_val)

        trial.report(min(loss_vals), epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return min(loss_vals)

In [10]:
sampler = TPESampler(seed=params_FNO['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-04 11:46:56,607] A new study created in memory with name: no-name-ed42057c-54b8-4327-9c2f-74e2784155d6


In [11]:
study.optimize(objective, n_trials=20, timeout=6000)

[I 2023-08-04 12:02:29,396] Trial 0 finished with value: 1.999834410071344e-06 and parameters: {'n_fourier_layers': 4, 'lifting_layer': 41, 'Fourier_units_l0': 39, 'Fourier_modes_l0': 14, 'Fourier_units_l1': 55, 'Fourier_modes_l1': 14, 'Fourier_units_l2': 64, 'Fourier_modes_l2': 15, 'Fourier_units_l3': 47, 'Fourier_modes_l3': 13, 'optimizer': 'RMSprop', 'lr': 1.7326529663635617e-05, 'batch_size': 121}. Best is trial 0 with value: 1.999834410071344e-06.
[I 2023-08-04 12:18:41,548] Trial 1 finished with value: 6.443213074192045e-08 and parameters: {'n_fourier_layers': 4, 'lifting_layer': 38, 'Fourier_units_l0': 37, 'Fourier_modes_l0': 14, 'Fourier_units_l1': 49, 'Fourier_modes_l1': 15, 'Fourier_units_l2': 60, 'Fourier_modes_l2': 15, 'Fourier_units_l3': 52, 'Fourier_modes_l3': 15, 'optimizer': 'RMSprop', 'lr': 0.0001495740267170668, 'batch_size': 173}. Best is trial 1 with value: 6.443213074192045e-08.
[W 2023-08-04 12:23:17,478] Trial 2 failed with parameters: {'n_fourier_layers': 1, 'li

KeyboardInterrupt: 

In [12]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  2
Best trial:
  Value:  6.443213074192045e-08
  Params: 
    n_fourier_layers: 4
    lifting_layer: 38
    Fourier_units_l0: 37
    Fourier_modes_l0: 14
    Fourier_units_l1: 49
    Fourier_modes_l1: 15
    Fourier_units_l2: 60
    Fourier_modes_l2: 15
    Fourier_units_l3: 52
    Fourier_modes_l3: 15
    optimizer: RMSprop
    lr: 0.0001495740267170668
    batch_size: 173


In [ ]:
modes = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'Fourier':
        if key.split('_')[1] == 'modes':
            modes.append(value)
        if key.split('_')[1] == 'units':
            layer_dims.append(value)
    elif key.split('_')[0] == 'lifting':
        lifting = value
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value
    elif key == 'batch_size':
        batch_size = value

Use this to fill config_step_1.py

In [ ]:
print(layer_dims, '\n', modes, '\n', lifting, '\n', optimizer, '\n', lr, '\n', batch_size)